<a href="https://colab.research.google.com/github/spencer18001/machine-learning-zoomcamp/blob/main/08/hw_08.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install tensorflow=2.17.1
!wget https://github.com/SVizor42/ML_Zoomcamp/releases/download/straight-curly-data/data.zip
!unzip data.zip

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

In [3]:
inputs = keras.Input(shape=(200, 200, 3))
x = keras.layers.Conv2D(32, kernel_size=(3, 3), activation="relu")(inputs)
x = keras.layers.MaxPooling2D(pool_size=(2, 2))(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(64, activation="relu")(x)
outputs = keras.layers.Dense(1, activation="sigmoid")(x)
model = keras.models.Model(inputs, outputs)

optimizer = keras.optimizers.SGD(learning_rate=0.002, momentum=0.8)
model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"]) # Q1

model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 200, 200, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d (Conv2D)                      │ (None, 198, 198, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 99, 99, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 313632)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 64)                  │      20,072,512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 20,073,473 (76.57 MB)

 Trainable params: 20,073,473 (76.57 MB)

 Non-trainable params: 0 (0.00 B)

In [4]:
model.count_params() # Q2

20073473

In [5]:
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    "data/train",
    target_size=(200, 200),
    class_mode="binary",
    batch_size=20,
    shuffle=True
)
test_generator = test_datagen.flow_from_directory(
    "data/test",
    target_size=(200, 200),
    class_mode="binary",
    batch_size=20,
    shuffle=True
)

Found 800 images belonging to 2 classes.
Found 201 images belonging to 2 classes.


In [6]:
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=test_generator
)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


40/40 ━━━━━━━━━━━━━━━━━━━━ 38s 886ms/step - accuracy: 0.5493 - loss: 0.7514 - val_accuracy: 0.5771 - val_loss: 0.6701
Epoch 2/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 40s 869ms/step - accuracy: 0.6305 - loss: 0.6533 - val_accuracy: 0.6119 - val_loss: 0.6529
Epoch 3/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 41s 880ms/step - accuracy: 0.6238 - loss: 0.6252 - val_accuracy: 0.6169 - val_loss: 0.6374
Epoch 4/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 35s 858ms/step - accuracy: 0.6619 - loss: 0.5982 - val_accuracy: 0.6418 - val_loss: 0.6190
Epoch 5/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 37s 897ms/step - accuracy: 0.6630 - loss: 0.5895 - val_accuracy: 0.6169 - val_loss: 0.6417
Epoch 6/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 35s 851ms/step - accuracy: 0.6921 - loss: 0.5813 - val_accuracy: 0.5871 - val_loss: 0.6808
Epoch 7/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 36s 853ms/step - accuracy: 0.6845 - loss: 0.5782 - val_accuracy: 0.6766 - val_loss: 0.6062
Epoch 8/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 41s 860ms/step - accuracy: 0.7464 - loss: 0.5105 - val_accuracy: 0.646

In [7]:
np.median(history.history["accuracy"]) # Q3

0.6918749809265137

In [8]:
np.std(history.history["loss"]) # Q4

0.06114903130736359

In [9]:
augmented_train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=50,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode="nearest"
)

augmented_train_generator = augmented_train_datagen.flow_from_directory(
    "data/train",
    target_size=(200, 200),
    class_mode="binary",
    batch_size=20,
    shuffle=True
)

Found 800 images belonging to 2 classes.


In [10]:
history_aug = model.fit(
    augmented_train_generator,
    epochs=10,
    validation_data=test_generator
)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


40/40 ━━━━━━━━━━━━━━━━━━━━ 42s 1s/step - accuracy: 0.6379 - loss: 0.6276 - val_accuracy: 0.6716 - val_loss: 0.5866
Epoch 2/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 81s 962ms/step - accuracy: 0.6411 - loss: 0.6298 - val_accuracy: 0.6667 - val_loss: 0.5840
Epoch 3/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 82s 994ms/step - accuracy: 0.6768 - loss: 0.5897 - val_accuracy: 0.6716 - val_loss: 0.6182
Epoch 4/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 82s 955ms/step - accuracy: 0.6709 - loss: 0.6054 - val_accuracy: 0.6915 - val_loss: 0.5687
Epoch 5/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 41s 946ms/step - accuracy: 0.6921 - loss: 0.5896 - val_accuracy: 0.6965 - val_loss: 0.5708
Epoch 6/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 83s 996ms/step - accuracy: 0.6581 - loss: 0.6222 - val_accuracy: 0.6667 - val_loss: 0.6043
Epoch 7/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 44s 1s/step - accuracy: 0.6954 - loss: 0.5760 - val_accuracy: 0.6965 - val_loss: 0.5649
Epoch 8/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 42s 981ms/step - accuracy: 0.6880 - loss: 0.6055 - val_accuracy: 0.6866 - va

In [11]:
np.mean(history_aug.history["val_loss"]) # Q5

0.5820859193801879

In [12]:
np.mean(history_aug.history["val_accuracy"][-5:]) # Q6

0.6905472636222839